# Advent of Code 2025 Solutions

# [Day 1: Secret Entrance](https://adventofcode.com/2025/day/1)

## Part 1

We just need to keep track of where our `dial` is pointing. Since the dial is a circle, we can look at the remainder $\mod{100}$ everytime we rotate our dial by a certain amount. Every time our dial points at `0` after a rotation, we add one to our counter.

In [1]:
with open('input1.txt', 'r') as file:
    input_text = file.read()

In [2]:
def get_rotations(input_text):
    lines = input_text.strip().splitlines()
    return lines

In [3]:
rotations = get_rotations(input_text)
dial = 50
zero_counter = 0
for rotation in rotations:
    direction = rotation[0]
    distance = int(rotation[1:])
    shift = distance if direction == 'R' else - distance
    dial = (dial + shift) % 100
    if dial == 0:
        zero_counter += 1

In [4]:
zero_counter

1177

## Part 2

For this part, we add or subtract the appropriate distance based on the direction of our rotation and keep track of this value *without* modding by 100. Let `start` be the state of our dial before a rotation and `end` be the state of our dial after a rotation. Then the number of clicks that occur during this rotation corresponds to the number of multiples of 100 between our `start` and `end`. We can count this in most cases with $$\left \vert \left \lfloor \frac{\textrm{start}}{100} \right\rfloor - \left\lfloor \frac{\textrm{end}}{100} \right\rfloor \right \vert.$$

The caveat is that if we are rotating left, and we start on a multiple of 100, this method will count an additional click that we do not want to count. For example, if `start = 100` and our rotation is `L5` to get `end = 95`, this corresponds to our `dial` starting at `0` and ending at `95`. We do not want count a click in this rotation, but $$\left \lfloor \frac{100}{100} \right\rfloor - \left\lfloor \frac{95}{100} \right\rfloor = 1 - 0 = 1,$$
so our initial counting method would count 1 click during this rotation.

Similarly, if we are rotating left, and we end on a multiple of 100, we want to count this end state as a click even though `floor(start/100) - floor(end/100)` will not detect this. For example, consider `start = 340` with rotation `L40` to give us `end = 200`. This corresponds to our `dial` starting at `40`, wrapping all the way around once, and then rotating further to end at `0`. This means we should count `2` clicks in this rotation, but $$\left \lfloor \frac{340}{100} \right\rfloor - \left\lfloor \frac{200}{100} \right\rfloor = 3 - 2 = 1.$$ To fix both of these edge cases, we add in two if statements to adjust the number of clicks in a left-rotation accordingly.

In [5]:
dial = 50
click_counter = 0
for rotation in rotations:
    direction = rotation[0]
    distance = int(rotation[1:])
    
    if direction == 'R':
        start = dial
        dial += distance
        end = dial
    else:
        start = dial
        if start % 100 == 0:
            start -= 1
        dial -= distance
        end = dial
        if end % 100 == 0:
            end -= 1
    click_counter += abs(start//100 - end//100)

In [6]:
click_counter

6768